# 🧪 TPC-DS Benchmark Queries 6–10
Databricks SQL using `%run ./specify_catalog_schema`

In [0]:
%run ./specify_catalog_schema

In [0]:
%sql
-- Query 6: Price-Sensitive Customers in TN
-- TPC-DS Query 6
SELECT
    a.ca_state,
    COUNT(*) AS cnt
FROM customer_address a,
     customer c,
     store_sales s,
     date_dim d
WHERE a.ca_address_sk = c.c_current_addr_sk
  AND c.c_customer_sk = s.ss_customer_sk
  AND s.ss_sold_date_sk = d.d_date_sk
  AND d.d_year = 2000
  AND a.ca_state = 'TN'
GROUP BY a.ca_state
LIMIT 100;

In [0]:
%sql
-- Query 7: Top Brands by Revenue
-- TPC-DS Query 7
SELECT
    i_item_id,
    i_item_desc,
    i_current_price,
    i_class,
    i_category,
    i_manufact_id,
    i_manufact,
    SUM(ss_ext_sales_price) AS itemrevenue,
    SUM(ss_ext_sales_price) * 100 / SUM(SUM(ss_ext_sales_price)) OVER (PARTITION BY i_class) AS revenueratio
FROM store_sales, item, date_dim
WHERE ss_item_sk = i_item_sk
  AND ss_sold_date_sk = d_date_sk
  AND d_date BETWEEN DATE '2000-01-27' AND DATE '2000-02-27'
GROUP BY i_item_id, i_item_desc, i_current_price,
         i_class, i_category, i_manufact_id, i_manufact
ORDER BY i_item_id
LIMIT 100;

In [0]:
%sql
-- Query 8: Revenue Comparison Over Time
-- TPC-DS Query 8 (Fixed)
WITH avg_category_price AS (
    SELECT
        i_category,
        AVG(i_current_price) AS avg_price
    FROM item
    GROUP BY i_category
)
SELECT
    c_last_name,
    c_first_name,
    s_store_name,
    ca_state,
    i.i_item_desc,
    ss_ticket_number,
    ss_quantity,
    ss_wholesale_cost,
    ss_sales_price
FROM store_sales ss
JOIN customer c ON ss.ss_customer_sk = c.c_customer_sk
JOIN customer_address ca ON c.c_current_addr_sk = ca.ca_address_sk
JOIN store s ON ss.ss_store_sk = s.s_store_sk
JOIN item i ON ss.ss_item_sk = i.i_item_sk
JOIN date_dim d ON ss.ss_sold_date_sk = d.d_date_sk
JOIN avg_category_price acp ON i.i_category = acp.i_category
WHERE d.d_month_seq BETWEEN 1200 AND 1211
  AND i.i_current_price > 1.2 * acp.avg_price
  AND i.i_manufact_id IN (977, 998, 980, 964)
  AND ca.ca_state IN ('TN')
ORDER BY c.c_last_name, c.c_first_name, s.s_store_name, ca.ca_state
LIMIT 100;

In [0]:
%sql
-- Query 9: Frequent Buyers with High Spend
-- TPC-DS Query 9
SELECT
    c_last_name,
    c_first_name,
    c_salutation,
    c_preferred_cust_flag,
    ss_ticket_number,
    cnt
FROM customer, store_sales,
     (SELECT ss_ticket_number AS tkt,
             ss_customer_sk AS csk,
             COUNT(*) AS cnt
      FROM store_sales
      GROUP BY ss_ticket_number, ss_customer_sk
      HAVING COUNT(*) > 1) sls
WHERE ss_ticket_number = tkt
  AND ss_customer_sk = csk
  AND ss_customer_sk = c_customer_sk
ORDER BY cnt DESC, c_last_name, c_first_name, ss_ticket_number
LIMIT 100;

In [0]:
%sql
-- Query 10: Sales Growth by Category
-- TPC-DS Query 10
SELECT
    i_category,
    i_class,
    i_brand,
    s_store_name,
    d_year,
    d_qoy,
    SUM(ss_sales_price) AS sum_sales,
    AVG(SUM(ss_sales_price)) OVER (PARTITION BY i_category) AS avg_quarterly_sales
FROM store_sales,
     date_dim,
     store,
     item
WHERE ss_sold_date_sk = d_date_sk
  AND ss_store_sk = s_store_sk
  AND ss_item_sk = i_item_sk
  AND d_year IN (1999, 2000, 2001)
GROUP BY i_category, i_class, i_brand, s_store_name, d_year, d_qoy
ORDER BY i_category, i_class, i_brand, s_store_name, d_year, d_qoy
LIMIT 100;